In [ ]:
import sys
import boto3
import botocore

# Define the parameters
BUCKET_TO_DEL = 'jobdetails-parquet-tbl-1'
DATABASE_TO_DEL = 'de_proj_database2'
TABLE_TO_DEL = 'jobdetails_jobs_with_details_parquet_tbl'
QUERY_OUTPUT_BUCKET = 's3://my-athena-bucket-dec2024/'

# Initialize S3 and Athena clients
s3_client = boto3.client('s3')
athena_client = boto3.client('athena')

# Function to empty the S3 bucket path for the table
def empty_s3_bucket(bucket_name, prefix):
    paginator = s3_client.get_paginator('list_objects_v2')
    operation_params = {'Bucket': bucket_name, 'Prefix': prefix}
    for page in paginator.paginate(**operation_params):
        if 'Contents' in page:
            delete_params = {'Objects': [{'Key': obj['Key']} for obj in page['Contents']]}
            s3_client.delete_objects(Bucket=bucket_name, Delete=delete_params)
            print(f"Deleted objects: {delete_params['Objects']}")

# Drop the Athena table
try:
    print(f"Dropping table {TABLE_TO_DEL} from database {DATABASE_TO_DEL}.")
    queryStart = athena_client.start_query_execution(
        QueryString=f"DROP TABLE IF EXISTS {TABLE_TO_DEL};",
        QueryExecutionContext={'Database': f'{DATABASE_TO_DEL}'},
        ResultConfiguration={'OutputLocation': f'{QUERY_OUTPUT_BUCKET}'}
    )

    # Poll the query execution status
    resp = ["FAILED", "SUCCEEDED", "CANCELLED"]
    response = athena_client.get_query_execution(QueryExecutionId=queryStart["QueryExecutionId"])
    while response["QueryExecution"]["Status"]["State"] not in resp:
        response = athena_client.get_query_execution(QueryExecutionId=queryStart["QueryExecutionId"])

    # Check the query status
    if response["QueryExecution"]["Status"]["State"] == 'FAILED':
        print("Failed to drop the table.")
        sys.exit(response["QueryExecution"]["Status"]["StateChangeReason"])
    else:
        print(f"Successfully dropped table: {TABLE_TO_DEL}")

    # Empty the S3 bucket for the table
    print(f"Cleaning up S3 data for bucket: {BUCKET_TO_DEL}")
    empty_s3_bucket(BUCKET_TO_DEL, TABLE_TO_DEL)

except botocore.exceptions.ClientError as error:
    print(f"An error occurred: {error}")
    sys.exit(1)

# Log the success
print(f"Successfully deleted table {TABLE_TO_DEL} and cleaned up associated S3 bucket data.")
